# MCBHSOLV program

Setting of the problem: compute solvation energy of a given molecule.

We need to compute following value:

\begin{equation}
\Delta G_{pol} = \frac{1}{2}\sum_i Q_i \int_{\Omega}
\frac{\sigma(\mathbf{r})}{\left| \mathbf{r-R_i} \right|} dS,
\end{equation}

where $Q_i$ is a charge of $i$-th atom in molecule,
$\mathbf{R_i}$ is a position vector of $i$-th atom in molecule,
$\Omega$ is a solvent excluded surface,
$\sigma$ is surface charge density,
$\mathbf{r}$ is a radius vector to surface.

Surface charge density is a solution of the following equation:

\begin{equation}
\sigma(\mathbf{r}) = \frac{1-\varepsilon}{2 \pi
(1+\varepsilon)}\left( \sum_i \frac{Q_i ((\mathbf{r-R_i})\cdot \mathbf{n})}{\left|
\mathbf{r-R_i} \right|^3} + \int_{\Omega}
\frac{\sigma(\mathbf{r'}) ((\mathbf{r-r'}) \cdot \mathbf{n})}{\left|
\mathbf{r-r'} \right|^3} dS' \right),
\end{equation}

where $\mathbf{n}$ is a perpendicular from surface to solvent,
$\varepsilon$ is a relative permittivity.

To find surface charge density, we approximate the surface by discrete elements with the Nystrom method for
the off-diagonal elements and the diagonal elements are computed from
the identities:

\begin{equation}
\begin{split}
& \int_{\Omega}\frac{((\mathbf{r-r'}) \cdot \mathbf{n'})}{\left|
\mathbf{r-r'} \right|^3} dS' = 2\pi, \\
& \int_{\Omega_{\varepsilon}} \frac{\sigma(\mathbf{r'}) ((\mathbf{r-r'}) \cdot \mathbf{n})}{\left|
\mathbf{r-r'} \right|^3} dS' \approx \sigma(\mathbf{r}) \left( 2 \pi - \int_{\Omega
\setminus \Omega_{\varepsilon}} \frac{((\mathbf{r-r'}) \cdot
\mathbf{n'})}{\left| \mathbf{r-r'} \right|^3} dS' \right).
\end{split}
\end{equation}

So, after discretization, we get matrix $A$ with following elements:

\begin{equation}
A_{ij} = \left\{ \begin{array}{ll}
\frac{(\varepsilon-1)}{4\pi(1+\varepsilon)} \frac{((\mathbf{r_i-r_j})
\cdot \mathbf{n_i}) S_i}{\left| \mathbf{r_i-r_j} \right|^3}, & i \ne j
\\ \frac{\varepsilon}{1+\varepsilon} -\sum_{k \ne j} A_{kj}, & i = j
\end{array} \right.,
\end{equation}

where $\mathbf{r_i}$ is a radius vector to center of $i$-th discrete element,
$\mathbf{n_i}$ is a perpendecular to $i$-th discrete element,
$S_i$ is an area of $i$-th discrete element.

After approximate solution is found, corresponding discretized solvation energy is computed.

In [1]:
# import all necessary modules and set parameters
from __future__ import print_function, absolute_import, division

from h2tools.collections import pcm

# set parameters
# eps is a relative permittivity,
# fname is a filename of surface,
# accuracy is an accuracy parameter for MCBH method
eps = 78.5
fname = '1c5y_fl_surface_and_atoms_0.10.txt'
accuracy = 1e-4

In [2]:
# Read data and find solvation energy iteratively
data = pcm.PCM_Data.from_file(fname, eps)
print('Computing solvation energy for surface \'{}\''.format(fname))
print('Solvation energy: {} kcal/mol\nIterations required: {}'.format(
        *data.find_solvation_energy(accuracy, onfly=0, verbose=1)))

Computing solvation energy for surface '1c5y_fl_surface_and_atoms_0.10.txt'
Cluster trees are generated in 0.418989896774 seconds
Depth level of each cluster tree: 11
Row cluster tree
    nodes : 999
    leaves: 500
Column cluster tree
    nodes : 999
    leaves: 500
Far-field interactions(MCBH method):
    Function calls: 16934
    Function values computed: 40957075
    Function time, seconds: 0.92
    Average time per function value, seconds: 2.24e-08
    Maxvol time, seconds: 6.04768228531
Near-field interactions:
    Function calls: 4082
    Function values computed: 1244929
    Function time, seconds: 0.07
    Average time per function value, seconds: 5.94e-08
Total time, seconds: 7.40
Memory:
    Basises, MB: 0.48
    Transfer matrices, MB: 7.70
    Far-field interactions, MB: 31.19
    Near-field interactions, MB: 10.61
Total memory, MB: 49.97
1 0.201430663433
2 0.0450155500923
3 0.00880750923907
4 0.00145186922517
5 0.00024052185881
6 3.26796442401e-05
Solvation energy: -66.731

In [3]:
# here we check approximation properties for matrix

from h2tools import Problem, ClusterTree
from h2tools.mcbh import mcbh

# set parameters
tau = 1e-4
block_size = 25
iters = 1
onfly = 0
verbose = 1

tree = ClusterTree(data, block_size)
problem = Problem(pcm.electro, tree, tree, symmetric=0, verbose=verbose)

# build matrix approximation
matrix = mcbh(problem, tau=tau, iters=iters, onfly=onfly, verbose=verbose)

# and check relative error of far field approximation (only works with ***pypropack*** installed)
print('Relative error of far-field approximation is {}'.format(matrix.diffnorm(far_only=1)))

matrix.svdcompress(1e-3, verbose=True)
print('Relative error of far-field approximation is {}'.format(matrix.diffnorm(far_only=1)))

matrix.svdcompress(1e-2, verbose=True)
print('Relative error of far-field approximation is {}'.format(matrix.diffnorm(far_only=1)))

Cluster trees are generated in 0.167444944382 seconds
Depth level of each cluster tree: 11
Row cluster tree
    nodes : 999
    leaves: 500
Column cluster tree
    nodes : 999
    leaves: 500
Far-field interactions(MCBH method):
    Function calls: 16934
    Function values computed: 40957075
    Function time, seconds: 0.90
    Average time per function value, seconds: 2.19e-08
    Maxvol time, seconds: 6.08900475502
Near-field interactions:
    Function calls: 4082
    Function values computed: 1244929
    Function time, seconds: 0.06
    Average time per function value, seconds: 5.13e-08
Total time, seconds: 7.42
Memory:
    Basises, MB: 0.48
    Transfer matrices, MB: 7.70
    Far-field interactions, MB: 31.19
    Near-field interactions, MB: 10.61
Total memory, MB: 49.97
Relative error of far-field approximation is 0.000496768717847
memory BEFORE SVD-compression: 49.97MB
memory AFTER SVD-compression: 17.17MB
recompression time: 2.23512601852
Relative error of far-field approximati

In [4]:
print(matrix)

H2matrix at 0x101a87f50
    Structure (h2/mcbh): h2
    Memory layout (low/full): full
    Shape: [8457, 8457]
    Total memory, MB: 14.99
        Basises, MB: 0.02
        Transfer matrices, MB: 1.63
        Far-field interactions, MB: 2.73
        Near-field interactions, MB: 10.61
